In this notebook we will export frequency differencing variables from dataflow

In [3]:
import numpy as np
import win32com.client as win32
import glob
import os

# Select platform

Sermilik cruise 2023

In [52]:
# Location of data files
ev_template = 'D:/Sermilik/NNA_EV/Sermilik_dBDiff.EV'

EV_folder = 'D:/Sermilik/NNA_EV/' 
data_folder = 'D:/Sermilik/NNA_EK80/NNA_EK80/'
export_folder = 'D:/Sermilik/NNA_dBDiff_Export/'

processed_export_folder = 'D:/Sermilik/images_clean/' 
processed_export_resample_folder = 'D:/Sermilik/images_resampled/' 

# Define variable name to export
#var_T1 = 'Calibration subset T1'
var_T2 = 'Calibration subset T2'
var_T3 = 'Calibration subset T3'
var_T4 = 'Calibration subset T4' #Passive
var_T5 = 'Calibration subset T5' #Passive
var_T6 = 'Calibration subset T6'
var_T7 = 'Calibration subset T7'#Passive

T_list = ['T2', 'T3', 'T4', 'T5', 'T6']

# Define variable name to export
#pvar_T1 = 'Processed data T1'  # 150 kHz
pvar_T2 = 'Processed data T2'   # 38 kHz
pvar_T3 = 'Processed data T3'   # 18 kHz
pvar_T4 = 'Processed data T4' #Passive  #70 kHz
pvar_T5 = 'Processed data T5' #Passive  #200 kHz 
pvar_T6 = 'Processed data T6'   # 120 kHz
#pvar_T7 = 'Processed data T7'#Passive  # 333 kHz



var_names = [var_T2,var_T3,var_T4,var_T5,var_T6]
processed_var_names = [pvar_T2,
                       pvar_T3,
                       pvar_T4, 
                       pvar_T5,
                       pvar_T6]
                       #pvar_T7]

Sv18_38 = 'S18-Sv38'
Sv38_70 = 'S38-Sv70'
Sv38_120 = 'S38-Sv120'
Sv38_200 = 'S38-Sv200'
Sv70_120 = 'S70-Sv120'
Sv70_200 = 'S70-Sv200'
Sv120_200 = 'S120-Sv200'

export_names = [Sv18_38, 
                Sv38_70, 
                Sv38_120, 
                Sv38_200, 
                Sv70_120, 
                Sv70_200,
                Sv120_200]

# Find all data files with extension .raw for the date frame
date = '20230806'
EV_file_name = sorted(glob.glob(EV_folder + f'Sermilik_dBDiff_{date}PM.EV'))
raw_file_names = sorted(glob.glob(data_folder + f'NNA_*{date}*.raw'))
cal_file=f'D:/Sermilik/NNA_EK80/NNA_EK80/Sermilik_{date}.ecs'

In [29]:
len(raw_file_names)

89

##### Export image


In [31]:
# Open new EV file

for ind in np.arange(0,1):
    # Connect to the Echoview application
    EvApp = win32.Dispatch("EchoviewCom.EvApplication") 
    #EvApp.Minimize()
    # Create new file
    EvFile = EvApp.NewFile(EV_file_names[ind])

    # Load data by chunks of 3 files (1 on period)
    Fileset = EvFile.Filesets.FindByName('Fileset 1')

    # Set cal file
    Fileset.SetCalibrationFile(cal_file)
    
    for sub_ind in range(0,81):
        Fileset.DataFiles.Add(raw_file_names[ind+sub_ind])

    i=0
    for T_i in T_list:
        
        # Select variable and recalculate
        Processed_Var = EvFile.Variables.FindByName(f'Processed data {T_i}')
        Sv_Var = EvFile.Variables.FindByName(f'Attenuated signal removal {T_i}')
        
        # Create a new editable line that offsets and spans gaps from bottom line
        #if i == 0:
        #    Offset_line = EvFile.Lines.FindByName('Bottom Smooth')
        #    New_Line = EvFile.Lines.CreateOffsetLinear(Offset_line, 1, -2, True)# Editable line with 2m offset from bottom, spans gaps
            
         #   Bubble_line = EvFile.Lines.FindByName('Bubble Offset')
         #   bubble_Line = EvFile.Lines.CreateOffsetLinear(Bubble_line, 1, 2, True)# Editable line with 2m offset from surface an bubble offset, spans gaps
            
        i += 1
        
        #Sv_Var.Properties.Analysis.ExcludeBelowLine = "Line 1" # Variable before processed data
        #Processed_Var.Properties.Analysis.ExcludeBelowLine = "Line 1"
        #Sv_Var.Properties.Analysis.ExcludeAboveLine = "Line 1 (1)"
        #Processed_Var.Properties.Analysis.ExcludeAboveLine = "Line 1 (1)"

        Processed_Var.Properties.Grid.SetDepthRangeGrid(1,50)
        Processed_Var.Properties.Grid.SetTimeDistanceGrid(2,1)

    # Close file
    #EvFile.Close()
    # Quit Echoview
    #EvApp.Quit()

In [33]:
EvApp = win32.Dispatch("EchoviewCom.EvApplication") 
EvFile = EvApp.NewFile()

<COMObject NewFile>

In [62]:
# close new file and work with previously opened file.

In [61]:
head, tailEv = os.path.split(EV_file_names[0])

for var_i in export_names:
    Sv_diff= EvFile.Variables.FindByName(var_i)
    
    Sv_diff.Properties.Grid.SetDepthRangeGrid(1,5)
    Sv_diff.Properties.Grid.SetTimeDistanceGrid(2,1)
    Sv_diff.ExportData(export_folder+tailEv[:-3]+var_i+'.csv',-1,-1)


In [48]:
Sv_18.ExportData()

AttributeError: 'NoneType' object has no attribute 'ExportData'

In [50]:
EvFile.Variables.FindByName('S18-Sv38')

<COMObject FindByName>

In [58]:
head, tailEv = os.path.split(EV_file_names[0])